Connect to drive

Sources:
https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34
https://www.analyticsvidhya.com/blog/2021/01/a-guide-to-the-naive-bayes-algorithm/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Applied Data Science/Thesis/Code'

/content/drive/MyDrive/Applied Data Science/Thesis/Code


Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score,KFold
from sklearn import naive_bayes
from sklearn.feature_extraction.text import CountVectorizer

Load csv files of retracted and non-retracted articles

In [ ]:
five_journal_train_test_data_set = pd.read_csv('/content/drive/MyDrive/Applied Data Science/Thesis/Code/Data (CSV)/five_journal_train_test_data_set.csv', encoding="utf-8-sig")
two_journal_external_data_set = pd.read_csv('/content/drive/MyDrive/Applied Data Science/Thesis/Code/Data (CSV)/two_journal_external_data_set.csv', encoding="utf-8-sig")

Classifier: Countvectorizer

In [ ]:
paper_sections = ['Title + Abstract PP S', 'Main content PP S', 'Discussion / Conclusion PP S', 'References PP S']

for section in paper_sections:
  print('\n########### ' + section + ": \n")

  if five_journal_train_test_data_set[section].isnull().values.any():
    nan_values = five_journal_train_test_data_set[five_journal_train_test_data_set[section].isnull()]
    five_journal_train_test_data_set = five_journal_train_test_data_set[~five_journal_train_test_data_set.ID.isin(nan_values.ID)]

  raw_X = list(five_journal_train_test_data_set[section].values) # the texts --> X
  X = []
  y = list(five_journal_train_test_data_set.Retracted.values) # the labels we want to predict --> Y

  for i in raw_X:
    if len(i.split(" ")) > 420:
      head = i.split(" ")[0:210]
      tail = i.split(" ")[-210:]
      headandtail = " ".join(head) + " ".join(tail) 
    else:
      headandtail = i
    X.append(headandtail)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.34, random_state=1)
  X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1)

  le = LabelEncoder()

  y_train = le.fit_transform(y_train)
  y_test = le.fit_transform(y_test)

  Tfidf_vect = TfidfVectorizer()
  Tfidf_vect.fit(X)
  Train_X_Tfidf = Tfidf_vect.transform(X_train)
  Test_X_Tfidf = Tfidf_vect.transform(X_test)

  Naive = naive_bayes.MultinomialNB()
  Naive.fit(Train_X_Tfidf,y_train)
  predictions_NB = Naive.predict(Test_X_Tfidf)

  print(classification_report(y_test, predictions_NB))

  print(confusion_matrix(y_test, predictions_NB))

  print("\n####### NOW RUNNING ON THE EXTERNAL DATA SET #########\n")

  if two_journal_external_data_set[section].isnull().values.any():
    nan_values = two_journal_external_data_set[two_journal_external_data_set[section].isnull()]
    two_journal_external_data_set = two_journal_external_data_set[~two_journal_external_data_set.ID.isin(nan_values.ID)]

  raw_X = list(two_journal_external_data_set[section].values) # the texts --> X
  X = []
  y = list(two_journal_external_data_set.Retracted.values) # the labels we want to predict --> Y

  for i in raw_X:
    if len(i.split(" ")) > 420:
      head = i.split(" ")[0:210]
      tail = i.split(" ")[-210:]
      headandtail = " ".join(head) + " ".join(tail) 
    else:
      headandtail = i
    X.append(headandtail)

  Test_X_Tfidf = Tfidf_vect.transform(X)

  predictions_NB = Naive.predict(Test_X_Tfidf)
  print(confusion_matrix(y, predictions_NB))

  print(classification_report(y, predictions_NB))


########### Title + Abstract PP S: 

              precision    recall  f1-score   support

           0       0.80      0.82      0.81        34
           1       0.78      0.75      0.76        28

    accuracy                           0.79        62
   macro avg       0.79      0.79      0.79        62
weighted avg       0.79      0.79      0.79        62

[[28  6]
 [ 7 21]]

####### NOW RUNNING ON THE EXTERNAL DATA SET #########

[[ 58  74]
 [ 11 121]]
              precision    recall  f1-score   support

           0       0.84      0.44      0.58       132
           1       0.62      0.92      0.74       132

    accuracy                           0.68       264
   macro avg       0.73      0.68      0.66       264
weighted avg       0.73      0.68      0.66       264


########### Main content PP S: 

              precision    recall  f1-score   support

           0       0.78      0.85      0.82        34
           1       0.80      0.71      0.75        28

    accurac